In [3]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("../jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [16]:
import requests
from bs4 import BeautifulSoup

label='rf_systems'
url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:'+ label
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text)

In [17]:
searchPage = True
count = 1

temp = []
user = {}
page = int('0')
  
while searchPage == True:
    for i in soup.find_all('div', class_ = 'gsc_1usr'):      
        #tag = []

        id = i.find('a')['href'].split('user=')[1]
        id.replace(" ", "_")
        temp.append(id)
        #temp['label'] = tag
        
    user['userID'] = temp
    try:
        #split nextpage after_author
        afterAuthor = soup.find('div', id = 'gsc_authors_bottom_pag').find('button', class_ = 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx')['onclick'].split('author\\x3d')[1].split('\\x26')[0]
        page = str(int(page) + 10) 
        
        url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:'+label+'&after_author='+afterAuthor+'&astart='+page
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text)
        
        count = count +1
        print(count)
    except KeyError as err:        
        searchPage == False
        break

            


AttributeError: 'NoneType' object has no attribute 'find'

In [4]:
import json
def jsontransfer(items):
    data = json.dumps(items)
    jsonStr = json.loads(data) 
    return jsonStr

In [38]:
def update_labelinfo(items, label):
    print(label)
    ref = db.collection(u'Label-Domain').document(label)
    ref.set(items)

In [21]:
updateLabel(user, label)

In [4]:
import requests
from bs4 import BeautifulSoup
import datetime
def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [14]:
def get_labelIDlist(label):

    url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:' + label
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
    }
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text)

    searchPage = True
    tempList = []
    Label = {}
    page = int('0')

    while searchPage == True:
        for i in soup.find_all('div', class_='gsc_1usr'):
            #tag = []

            id = i.find('a')['href'].split('user=')[1]
            tempList.append(id)
            #temp['label'] = tag

        Label['userID'] = tempList
        try:
            # split nextpage after_author
            afterAuthor = soup.find('div', id='gsc_authors_bottom_pag').find(
                'button', class_='gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx')['onclick'].split('author\\x3d')[1].split('\\x26')[0]
            page = str(int(page) + 10)

            url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:' + \
                label+'&after_author='+afterAuthor+'&astart='+page
            r = requests.get(url, headers=headers)
            soup = BeautifulSoup(r.text)
            print(page)
        except :
            searchPage == False
            break
    Label['updateTime'] = currentTime()
    return Label

In [ ]:
 t=get_LabelIDList('Bayesian_inference')
print(t)

In [22]:
print(type(t))

<class 'dict'>


In [47]:
updateLabel(t, '5G Wireless Communications')

In [9]:
def labelinfoformat(data):
    print(data)
    rewriteData = data
    if(len(data['userID']) == 0):
        print(len(data['userID']))
        return False
    # check updateTime format
    if(data['updateTime'] == '' or data['updateTime'] == None):
        rewriteData['personalData']['updateTime'] = currentTime()
        print(2)

    try:
        datetime.datetime.strptime(
            data['updateTime'], "%Y-%m-%d %H:%M:%S")
        print(1)
    except:
        rewriteData['updateTime'] = currentTime()
        print(3)
    return rewriteData

In [10]:

def LabelCrawl():
    print('label start')
    label = get_lastupdatelabel()
    label ='Bayesian_inference'# limit
    labellist = get_labelIDlist('Bayesian_inference')
    print(labellist)
    check_labelformat = labelinfoformat(labellist)

    # label list 為空或格式錯誤時回傳False,格式錯誤修正後回傳rewriteInfo
    if(check_labelformat != False):
        update_labelinfo(labellist, label)
        print('label 1done')
    else:
        rewrite_labelinfo = check_labelformat
        update_labelinfo(rewrite_labelinfo, label)
        print('label done')

In [11]:
def get_lastupdatelabel():
    query = db.collection(u'Label-Domain').where(u'updateTime', '==', None).limit(1)
    results = query.stream()
    for r in results:
        label = r.id
    return label

In [18]:
LabelCrawl()

label start
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
{'userID': ['ZyOio_YAAAAJ', 'CmWHB9gAAAAJ', 'W4SZGV8AAAAJ', 'L_HG640AAAAJ', 'aqgFQqMAAAAJ', 'rr8pZoUAAAAJ', 'k9SFchAAAAAJ', 'T1lNNLkAAAAJ', 'Y7MiQqkAAAAJ', '4wWQsj8AAAAJ', '2K3F0MMAAAAJ', 'odOmEY0AAAAJ', 'i_k7UrQAAAAJ', 'pw6xnRcAAAAJ', '0Xa0gGwAAAAJ', 'zN1unagAAAAJ', 'N0qz4VkAAAAJ', 'efOGg5IAAAAJ', 'WjCG3owAAAAJ', 'N3oasbQAAAAJ', 'H8fpNVYAAAAJ', 'lVpFF40AAAAJ', 'Va6LlhgAAAAJ', 'xz6_iTEAAAAJ', 'dDhxUmgAAAAJ', 'QrXr-4cAAAAJ', 'BAtd7FcAAAAJ', 'dgVYYngAAAAJ', 'vt1EAJcAAAAJ', 'HNhEAN8AAAAJ', '1gr9rvUAAAAJ', 'cqxv9PAAAAAJ', 'MUJ51_gAAAAJ', 'RDf-d68AAAAJ', 'KoJrMIAAAAAJ', 'Fd6mdHcAAAAJ', '33ynLk8AAAAJ', '4BEvaw8AAAAJ', 'Z-kAK98AAAAJ', '

NameError: name 'update_labelinfo' is not defined

In [71]:
id = 'pp'

In [72]:
id.replace(" ", "_")

'pp'